In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import requests

URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(URL)

In [3]:
soup = BeautifulSoup(page.text, 'html.parser')

In [4]:
datas = []

datas_elem = soup.find_all('tr')

for item in datas_elem:
    datas.append(item.text)
    
datas

['\nPostal Code\n\nBorough\n\nNeighbourhood\n',
 '\nM1A\n\nNot assigned\n\nNot assigned\n',
 '\nM2A\n\nNot assigned\n\nNot assigned\n',
 '\nM3A\n\nNorth York\n\nParkwoods\n',
 '\nM4A\n\nNorth York\n\nVictoria Village\n',
 '\nM5A\n\nDowntown Toronto\n\nRegent Park, Harbourfront\n',
 '\nM6A\n\nNorth York\n\nLawrence Manor, Lawrence Heights\n',
 "\nM7A\n\nDowntown Toronto\n\nQueen's Park, Ontario Provincial Government\n",
 '\nM8A\n\nNot assigned\n\nNot assigned\n',
 '\nM9A\n\nEtobicoke\n\nIslington Avenue, Humber Valley Village\n',
 '\nM1B\n\nScarborough\n\nMalvern, Rouge\n',
 '\nM2B\n\nNot assigned\n\nNot assigned\n',
 '\nM3B\n\nNorth York\n\nDon Mills\n',
 '\nM4B\n\nEast York\n\nParkview Hill, Woodbine Gardens\n',
 '\nM5B\n\nDowntown Toronto\n\nGarden District, Ryerson\n',
 '\nM6B\n\nNorth York\n\nGlencairn\n',
 '\nM7B\n\nNot assigned\n\nNot assigned\n',
 '\nM8B\n\nNot assigned\n\nNot assigned\n',
 '\nM9B\n\nEtobicoke\n\nWest Deane Park, Princess Gardens, Martin Grove, Islington, Clover

In [5]:
k = []

for row in datas:
    k.append(row.split('\n'))
k

[['', 'Postal Code', '', 'Borough', '', 'Neighbourhood', ''],
 ['', 'M1A', '', 'Not assigned', '', 'Not assigned', ''],
 ['', 'M2A', '', 'Not assigned', '', 'Not assigned', ''],
 ['', 'M3A', '', 'North York', '', 'Parkwoods', ''],
 ['', 'M4A', '', 'North York', '', 'Victoria Village', ''],
 ['', 'M5A', '', 'Downtown Toronto', '', 'Regent Park, Harbourfront', ''],
 ['', 'M6A', '', 'North York', '', 'Lawrence Manor, Lawrence Heights', ''],
 ['',
  'M7A',
  '',
  'Downtown Toronto',
  '',
  "Queen's Park, Ontario Provincial Government",
  ''],
 ['', 'M8A', '', 'Not assigned', '', 'Not assigned', ''],
 ['',
  'M9A',
  '',
  'Etobicoke',
  '',
  'Islington Avenue, Humber Valley Village',
  ''],
 ['', 'M1B', '', 'Scarborough', '', 'Malvern, Rouge', ''],
 ['', 'M2B', '', 'Not assigned', '', 'Not assigned', ''],
 ['', 'M3B', '', 'North York', '', 'Don Mills', ''],
 ['', 'M4B', '', 'East York', '', 'Parkview Hill, Woodbine Gardens', ''],
 ['', 'M5B', '', 'Downtown Toronto', '', 'Garden District

### -The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
data = {'Postal Code': [],
       'Borough' : [],
       'Neighbourhood' : []}
data

{'Postal Code': [], 'Borough': [], 'Neighbourhood': []}

In [8]:
for i in range(1, len(k)):
    data['Postal Code'].append(k[i][1])
    data['Borough'].append(k[i][3])
    data['Neighbourhood'].append(k[i][5])

In [9]:
df = pd.DataFrame(data)
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
179,M9Z,Not assigned,Not assigned
180,,Canadian postal codes,
181,,,
182,NL,NS,PE


In [11]:
df = df.loc[0:179]

### - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned

In [12]:
df_assigned = df[df['Borough'] != 'Not assigned']
df_not_assigned = df[df['Borough'] == 'Not assigned']

In [13]:
df_assigned.reset_index(drop=True, inplace=True)

### - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
### These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [16]:
df_assigned.groupby('Postal Code').sum().reset_index()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


### - If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [17]:
df_not_assigned

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
7,M8A,Not assigned,Not assigned
10,M2B,Not assigned,Not assigned
15,M7B,Not assigned,Not assigned
...,...,...,...
174,M4Z,Not assigned,Not assigned
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned


In [20]:
df_assigned.shape, df_not_assigned.shape

((103, 3), (77, 3))